# Documentation

`main.ipynb` is a Jupyter Notebook file that serves as the main entry point for this project. It contains both the code and the narrative that explains what the code does. For further reference, please check out the technical documentation of this tool, which can be found [here](/README.md).

This tool conducts the following analyses:

- speed data analysis
- c value modelling
- dynamic pressure modelling

To run this script, please use the following inputs:

- `INPUT_DIRECTORY` - valid path to a directory containing raw `.igc` files
- `OUTPUT_DIRECTORY` - valid path to an empty directory (output will be stored there)
- `FILE_EXTENSION` - set to `.igc`

# Imports

In [ ]:
import pandas as pd
from typing import List, Tuple
from datetime import datetime

import src.constants as constants
import src.helpers.file_processor as file_processor
import src.helpers.data_visualizer as data_visualizer
import src.algorithms.speed_analyzer as speed_analyzer
import src.algorithms.pressure_analyzer as pressure_analyzer
import src.algorithms.c_values_analyzer as c_values_analyzer

# Variables

In [ ]:
INPUT_DIRECTORY: str = "input_directory/" # end with "/
OUTPUT_DIRECTORY: str = "output_directory/" # end with /
FILE_EXTENSION: str = ".igc"

# Initialisation

In [ ]:
# objects
c_analyzer = c_values_analyzer.CAnalyzer()
file_processor = file_processor.FileProcessor()
speed_analyzer = speed_analyzer.SpeedAnalyzer()
data_visualizer = data_visualizer.DataVisualizer()
pressure_analyzer = pressure_analyzer.PressureAnalyzer()

# time
timestamp: str = datetime.now().strftime("%Y%m%d-%H%M%S")

# Data Pre-Processing

In [ ]:
# reference data
theoretical_reference: pd.DataFrame = pd.read_csv(f"{constants.THEORETICAL_REFERENCE_PATH}")
original_reference: pd.DataFrame = pd.read_csv(f"{constants.ORIGINAL_REFERENCE_PATH}")

print(f"Importing reference data:")
print(f"--> Theoretical reference: {constants.THEORETICAL_REFERENCE_PATH.split('/')[-1]}")
print(f"--> Original reference: {constants.ORIGINAL_REFERENCE_PATH.split('/')[-1]}")
print()

# experimental data
file_paths: List[str] = file_processor.get_file_paths(path=INPUT_DIRECTORY, file_extension=FILE_EXTENSION)

print(f"Importing experimental data:")
print(f"--> Found {len(file_paths)} files.")
print(f"--> The processing is initiated.")
print()

data_raw: pd.DataFrame = speed_analyzer.process_raw_data(file_paths=file_paths)

# Speed Data Analyis

In [ ]:
# filter and group data
theoretical_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=theoretical_reference, reference=True)
original_reference_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=original_reference, reference=True)
data_raw_filtered: pd.DataFrame = speed_analyzer.filter_raw_data(data=data_raw)

print(f"Filtered data:")
print(f"--> Data points for filtered theoretical reference: {len(theoretical_reference_filtered)} (lost {len(theoretical_reference) - len(theoretical_reference_filtered)})")
print(f"--> Data points for filtered original reference: {len(original_reference_filtered)} (lost {len(original_reference) - len(original_reference_filtered)})")
print(f"--> Data points for filtered tracklogs: {len(data_raw_filtered)} (lost {len(data_raw) - len(data_raw_filtered)})")
print()
print(f"Please note:")
print(f"--> Why are so many raw data points lost? This is due to the fact that the system filters out all data point that are not on a straight line. This is done to ensure that the data is as accurate as possible.")

# smooth experimental data
smoothed_data_raw: pd.DataFrame = speed_analyzer.savgol_filter(data=data_raw_filtered)
smoothed_data_grouped: pd.DataFrame = speed_analyzer.group_data(data=smoothed_data_raw)

print(f"Data smoothing:")
print(f"--> During smoothing, the raw data points were reduced from {len(data_raw_filtered)} to {len(smoothed_data_raw)} (lost {len(data_raw_filtered) - len(smoothed_data_raw)}).")
print(f"--> During grouping, the smoothed data points were reduced from {len(smoothed_data_raw)} to {len(smoothed_data_grouped)} (lost {len(smoothed_data_raw) - len(smoothed_data_grouped)}).")
print()

# export smoothed experimental data
print(f"Export data to CSV:")
print(f"--> Exporting smoothed data to {timestamp}_SJf_smoothed-experimental-speed-data_nicolas-huber.csv.")
print(f"--> Exporting grouped data to {timestamp}_SJf_experimental-speed-data_nicolas-huber.csv.")

speed_analyzer.export_to_csv(data=smoothed_data_raw, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_smoothed-experimental-speed-data_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=smoothed_data_grouped, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_grouped-experimental-speed-data_nicolas-huber.csv")

# data visualization
print(f"Data visualization:")
print(f"--> For reference, the theoretical and experimental speed data is visualized.")

data_visualizer.visualize_raw_speed_data(experimental_data=original_reference_filtered, theoretical_data=theoretical_reference_filtered, std_error=0.2, title="Experimentelle Geschwindigkeistdaten (original, 24.10.2022)")
data_visualizer.visualize_raw_speed_data(experimental_data=smoothed_data_grouped, theoretical_data=theoretical_reference_filtered, std_error=0.05, title="Experimentelle Geschwindigkeistdaten (optimiert, 31.03.2024)")

print(f"--> To visualize the quality increase of the data (original [10/24/2022] vs. updated [03/31/2024]), the following two plots are shown:")

speed_deviation_original: Tuple[float, float, float, float] = data_visualizer.visualize_speed_deviation(experimental_data=original_reference_filtered, theoretical_data=theoretical_reference_filtered, speed_analyzer=speed_analyzer, title="Abweichung der experimentellen Geschwindigkeitsdaten (original, 24.10.2022)")
speed_deviation_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_speed_deviation(experimental_data=smoothed_data_grouped, theoretical_data=theoretical_reference_filtered, speed_analyzer=speed_analyzer, title="Abweichung der experimentellen Geschwindigkeitsdaten (optimiert, 31.03.2024)")

# report
score_original: float = speed_analyzer.score_stats(speed_deviation_original)
score_optimized: float = speed_analyzer.score_stats(speed_deviation_optimized)

print(f"Report:")
speed_analyzer.print_report(score_original=score_original, score_optimized=score_optimized, deviation_original=speed_deviation_original, deviation_optimized=speed_deviation_optimized, datasets=[original_reference, original_reference_filtered, data_raw, smoothed_data_grouped])

# C Value Modelling

In [ ]:
# pre-process datasets
print(f"Manipulating datasets:")
print(f"--> The datasets are being extended:")
print(f"----> The original reference dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"----> The theoretical reference dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"----> The experimental dataset is being extended with the airspeed, that's calculated based on horizontal and vertical speed.")
print(f"--> Vertical speeds are converted to be positive:")
print(f"----> The vertical speeds of the original reference dataset are being converted to be positive.")
print(f"----> The vertical speeds of the theoretical reference dataset are being converted to be positive.")
print(f"----> The vertical speeds of the experimental dataset are being converted to be positive.")

airspeed_original_reference: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=original_reference_filtered))
airspeed_theoretical_reference: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=theoretical_reference_filtered))
airspeed_smoothed_data_grouped: pd.DataFrame = c_analyzer.positive_vertical_speed(speed_data=c_analyzer.calculate_airspeed(speed_data=smoothed_data_grouped))

print(f"--> The datasets have been extended and manipulated.")
print()

# model c values
print(f"C values:")
print(f"--> Calculating the c values for the following conditions:")
print(f"----> Altitude: {constants.ALTITUDE} m")
print(f"----> Air density: {constants.AIR_DENSITY} kg/m^3")
print(f"----> Gravity: {constants.GRAVITY} m/s^2")
print(f"----> Wing area: {constants.WING_AREA} m^2")
print(f"----> Takeoff mass: {constants.MASS} kg")
print(f"--> Calculating the c values for the following datasets:")
print(f"----> The c values are being calculated for the original reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the theoretical reference dataset (simplified & optimized algorithm).")
print(f"----> The c values are being calculated for the experimental dataset (simplified & optimized algorithm).")

c_values_original_reference_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_original_reference)
c_values_original_reference_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_original_reference, algorithm=True)
c_values_theoretical_reference_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_theoretical_reference)
c_values_theoretical_reference_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_theoretical_reference, algorithm=True)
c_values_experimental_simplified: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped)
c_values_experimental_optimized: pd.DataFrame = c_analyzer.process_c_values(speed_data=airspeed_smoothed_data_grouped, algorithm=True)

print(f"--> The c values have been calculated.")
print()

# export data
print(f"Export data to CSV:")
print(f"--> The original reference dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_flight-analyzer_c-values-original-reference_simplified_nicolas-huber.csv")
print(f"--> The original reference dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_flight-analyzer_c-values-original-reference_optimized_nicolas-huber.csv")
print(f"--> The theoretical reference dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_flight-analyzer_c-values-theoretical-reference_simplified_nicolas-huber.csv")
print(f"--> The theoretical reference dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_flight-analyzer_c-values-theoretical-reference_optimized_nicolas-huber.csv")
print(f"--> The experimental dataset is being exported to the output directory (simplified algorithm): {timestamp}_SJf_flight-analyzer_c-values-experimental_simplified_nicolas-huber.csv")
print(f"--> The experimental dataset is being exported to the output directory (optimized algorithm): {timestamp}_SJf_flight-analyzer_c-values-experimental_optimized_nicolas-huber.csv")
print()

speed_analyzer.export_to_csv(data=c_values_original_reference_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-original-reference_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_original_reference_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-original-reference_optimized_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_theoretical_reference_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-theoretical-reference_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_theoretical_reference_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-theoretical-reference_optimized_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_experimental_simplified, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-experimental_simplified_nicolas-huber.csv")
speed_analyzer.export_to_csv(data=c_values_experimental_optimized, file_path=f"{OUTPUT_DIRECTORY}/{timestamp}_SJf_flight-analyzer_c-values-experimental_optimized_nicolas-huber.csv")

# data visualization
print(f"Data visualization:")
print(f"--> For reference, the c values modelled based on the theoretical data are plotted.")
print(f"----> The c value model for the simplified algorithm (theoretical reference).")

data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title="Theoretische Ca-Werte (vereinfachter c-Algorithmus)")
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title="Theoretische Cw-Werte (vereinfachter c-Algorithmus)")

print(f"----> The c value model for the optimized algorithm (theoretical reference).")

data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title="Theoretische Ca-Werte (optimierter c-Algorithmus)")
data_visualizer.visualize_c_values_theoretical(theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title="Theoretische Cw-Werte (optimierter c-Algorithmus)")

print(f"--> For reference, the c value models for the simplified algorithm are plotted:")
print(f"----> Modelled c values for original reference dataset (simplified algorithm).")

data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (vereinfachter c-Algorithmus, originale Daten [24.10.2022])')
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (vereinfachter c-Algorithmus, originale Daten [24.10.2022])')

print(f"---> Deviation of modelled c values for original reference dataset (simplified algorithm).")

deviation_ca_original_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Abweichung der Ca-Werte (vereinfachter c-Algorithmus, originale Daten [24.10.2022])')
deviation_cw_original_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Abweichung der Cw-Werte (vereinfachter c-Algorithmus, originale Daten [24.10.2022])')

print(f"----> Modelled c values for experimental dataset (simplified algorithm).")

data_visualizer.visualize_c_values(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (vereinfachter c-Algorithmus, optimierte Daten [31.03.2024])')
data_visualizer.visualize_c_values(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (vereinfachter c-Algorithmus, optimierte Daten [31.03.2024])')

print(f"---> Deviation of modelled c values for experimental dataset (simplified algorithm).")

deviation_ca_experimental_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Ca [0.5]', title='Abweichung der Ca-Werte (vereinfachter c-Algorithmus, optimierte Daten [31.03.2024])')
deviation_cw_experimental_simplified: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_simplified, theoretical_data=c_values_theoretical_reference_simplified, key='Cw [0.5]', title='Abweichung der Cw-Werte (vereinfachter c-Algorithmus, optimierte Daten [31.03.2024])')

print(f"--> For reference, the c value models for the optimized algorithm are plotted:")
print(f"----> Modelled c values for original reference dataset (optimized algorithm).")

data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (optimierter c-Algorithmus, originale Daten [24.10.2022])')
data_visualizer.visualize_c_values(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (optimierter c-Algorithmus, originale Daten [24.10.2022])')

print(f"---> Deviation of modelled c values for original reference dataset (optimized algorithm).")

deviation_ca_original_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Abweichung der Ca-Werte (optimierter c-Algorithmus, originale Daten [24.10.2022])')
deviation_cw_original_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_original_reference_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Abweichung der Cw-Werte (optimierter c-Algorithmus, originale Daten [24.10.2022])')

print(f"----> Modelled c values for experimental dataset (optimized algorithm).")

data_visualizer.visualize_c_values(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Experimentelle vs. theoretische Ca-Werte (optimierter c-Algorithmus, optimierte Daten [31.03.2024])')
data_visualizer.visualize_c_values(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Experimentelle vs. theoretische Cw-Werte (optimierter c-Algorithmus, optimierte Daten [31.03.2024])')

print(f"---> Deviation of modelled c values for experimental dataset (optimized algorithm).")

deviation_ca_experimental_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Ca [0.5]', title='Abweichung der Ca-Werte (optimierter c-Algorithmus, optimierte Daten [31.03.2024])')
deviation_cw_experimental_optimized: Tuple[float, float, float, float] = data_visualizer.visualize_c_values_deviation(experimental_data=c_values_experimental_optimized, theoretical_data=c_values_theoretical_reference_optimized, key='Cw [0.5]', title='Abweichung der Cw-Werte (optimierter c-Algorithmus, optimierte Daten [31.03.2024])')

# report
score_original_ca_simplified: float = c_analyzer.score_stats(stats=deviation_ca_original_simplified)
score_original_cw_simplified: float = c_analyzer.score_stats(stats=deviation_cw_original_simplified)
score_experimental_ca_simplified: float = c_analyzer.score_stats(stats=deviation_ca_experimental_simplified)
score_experimental_cw_simplified: float = c_analyzer.score_stats(stats=deviation_cw_experimental_simplified)
score_original_ca_optimized: float = c_analyzer.score_stats(stats=deviation_ca_original_optimized)
score_original_cw_optimized: float = c_analyzer.score_stats(stats=deviation_cw_original_optimized)
score_experimental_ca_optimized: float = c_analyzer.score_stats(stats=deviation_ca_experimental_optimized)
score_experimental_cw_optimized: float = c_analyzer.score_stats(stats=deviation_cw_experimental_optimized)

c_analyzer.print_report(score_original_ca_simplified=score_original_ca_simplified, score_original_cw_simplified=score_original_cw_simplified, score_experimental_ca_simplified=score_experimental_ca_simplified, score_experimental_cw_simplified=score_experimental_cw_simplified, score_original_ca_optimized=score_original_ca_optimized, score_original_cw_optimized=score_original_cw_optimized, score_experimental_ca_optimized=score_experimental_ca_optimized, score_experimental_cw_optimized=score_experimental_cw_optimized)

# Dynamic Pressure Modelling

In [ ]:
# model pressure
print(f"Pressure analysis:")
print(f"--> The pressure data is being processed for the theoretical reference dataset.")
print(f"--> The pressure data is being processed for the experimental dataset.")
print()

pressure_data_theoretical: pd.DataFrame = pressure_analyzer.process_pressure_data(data=c_values_theoretical_reference_optimized)
pressure_data_experimental: pd.DataFrame = pressure_analyzer.process_pressure_data(data=c_values_experimental_optimized)

# export pressure data
print(f"Export to CSV:")
print(f"The theoretical reference dataset is being exported to a CSV file: {OUTPUT_DIRECTORY}{timestamp}_SJf_flight-analyzer_dynamic-pressure_theoretical-reference_nicolas-huber.csv")
print(f"The experimental dataset is being exported to a CSV file: {OUTPUT_DIRECTORY}{timestamp}_SJf_flight-analyzer_dynamic-pressure_experimental_nicolas-huber.csv")

pressure_analyzer.export_to_csv(data=pressure_data_theoretical, path=f"{OUTPUT_DIRECTORY}{timestamp}_SJf_flight-analyzer_dynamic-pressure_theoretical-reference_nicolas-huber.csv")
pressure_analyzer.export_to_csv(data=pressure_data_experimental, path=f"{OUTPUT_DIRECTORY}{timestamp}_SJf_flight-analyzer_dynamic-pressure_experimental_nicolas-huber.csv")

# data visualization
print(f"Data visualization:")
print(f"--> The dynamic pressure data for the theoretical reference dataset is being visualized against the experimental dataset.")

data_visualizer.visualize_pressure(experimental_data=pressure_data_experimental, theoretical_data=pressure_data_theoretical, title="Experimentelle Modellierung des Staudrucks am Flügel")
data_visualizer.visualize_pressure_deviation(experimental_data=pressure_data_experimental, theoretical_data=pressure_data_theoretical, title="Abweichung des Staudrucks am Flügel")

# System Info

In [ ]:
print(f"@ Author {constants.AUTHOR}")
print(f"@ Author Email {constants.AUTHOR_EMAIL}")
print(f"@ Author URL {constants.AUTHOR_URL}")
print(f"@ GitHub URL {constants.GITHUB_URL}")